# Textklassifikation mit Naive Bayes

Der Gemeindechor von Hinterspiessen geht alljährlich auf einen Wanderausflug. Findet er auch dieses Jahr statt? Wir beantworten diese Frage mit einem *Naive-Bayes-Klassifikator*.
Lernziele:
- Was sind bedingte Wahrscheinlichkeiten? Sie lernen, diese selber zu berechnen.
- Wie funktioniert ein Naive-Bayes-Klassifikator? Sie lernen den Satz von Bayes und sehen, wie der Naive-Bayes-Klassifikator darauf aufbaut. 
- Was ist daran "naiv"? Wir besprechen den Begriff der statistischen Unabhängigkeit von Features und sehen, wie dies den Bayes-Klassifikator effizient macht. Eine kleine Überdosis an Formeln gewöhnt Sie an den Umgang mit Ausdrücken wie "$p(x|y)$"...

Die Wettervorhersagen sind schon bekannt, der Entscheid des Chorleiters steht noch aus. In den Vergangenen Jahren galt:

In [2]:
import pandas as pd
#Anzahl Kommastellen bei floating-point-Ausgabe:
%precision 3

'%.3f'

In [4]:
#Wir laden die Daten:
df = pd.read_csv('../Materialien/datasets/Ausflugswetter.csv')

In [5]:
df

,Aussicht,Temperatur,Feuchtigkeit,Wind,Ausflug findet statt
0,Sonne,Heiss,Hoch,FALSCH,Nein
1,Sonne,Heiss,Hoch,WAHR,Nein
2,Bewölkt,Heiss,Hoch,FALSCH,Ja
3,Regen,Mild,Hoch,FALSCH,Ja
4,Bewölkt,Mild,Hoch,FALSCH,Ja
5,Bewölkt,Kühl,Normal,WAHR,Ja
6,Sonne,Mild,Normal,WAHR,Nein
7,Sonne,Mild,Normal,FALSCH,Ja
8,Sonne,Mild,Normal,FALSCH,Ja
9,Regen,Kühl,Normal,WAHR,Ja


In [4]:
#Benenne Zielvariable um- nur zur Bequemlichkeit 
#(df.Ausflug findet statt ist keine korrekte Syntax, df.y schon):
df.columns=df.columns[:-1].tolist()+['y']
df.head(3)

,Aussicht,Temperatur,Feuchtigkeit,Wind,y
0,Sonne,Heiss,Hoch,FALSCH,Nein
1,Sonne,Heiss,Hoch,WAHR,Nein
2,Bewölkt,Heiss,Hoch,FALSCH,Ja


## Bedingte Wahrscheinlichkeiten

Bedingte Wahrscheinlichkeiten ergeben sich, wenn die Wahrscheinlichkeit von Ereignissen voneinander abhängen. Eine Untersuchung (Daten frei erfunden!) zeigt z.B. 

||Mammografie diagnostiziert Brustkrebs|Mammografie findet keinen Brustkrebs|
|---------------------------------------------------------------------------|
|Patientin entwickelt Brustkrebs|2000|700|
|Patientin entwickelt keinen Brustkrebs|200|10000|
Die Brustkrebswahrscheinlichkeit wäre hier $$p(\text{Brustkrebs})=\frac{2000+700}{2000+700+200+10000}=\frac{2700}{12900}$$ und bezieht sich auf die Gesamtpopulation.  

**Aufgabe 1** Bestimmen Sie die folgenden Wahrscheinlichkeiten:
1. $P(Temperatur=Heiss)=$  
2. $P(c=Ja)=$
3. ${\arg \max}_c P(c)=$  

Die Klassen c sind c=Nein und c=Ja und beziehen sich darauf, ob der Ausflug stattfindet. 

In [5]:
#Lösung 1:
(df.Temperatur=='Heiss').sum()/(df.Temperatur=='Heiss').size

0.286

In [6]:
#Lösung 2:
(df.y=='Ja').sum()/df.y.size

0.643

In [7]:
#Lösung 3:
(df.y=='Ja').sum(),(df.y=='Nein').sum()

(9, 5)

Also ist $c=Ja$ die Antwort. "Ja" kommt häufiger vor als "Nein".

Nochmals die Tabelle zur Erinnerung:

||Mammografie diagnostiziert Brustkrebs|Mammografie findet keinen Brustkrebs|
|---------------------------------------------------------------------------|
|Patientin entwickelt Brustkrebs|2000|700|
|Patientin entwickelt keinen Brustkrebs|200|10000|

Eine bedingte Wahrscheinlichkeit ist nun ein Verhältnis, welches im Nenner nicht die Gesamtpopulation (hier alle 2000+700+200+10000=12900 Frauen) enthält, sondern nur jene, welche die Bedingung erfüllen: z.B. die Precision

$$p(\text{Patientin entwickelt Brustkrebs} \,|\, \text{Mammografie diagnostiziert Brustkrebs})=\frac{\#(\text{Patientin entwickelt diagnostizierten Brustkrebs})}{\#(\text{Mammografie diagnostiziert Brustkrebs})}=\frac{2000}{2200}$$
Oder auch den Recall

$$p(\text{Mammografie diagnostiziert Brustkrebs} \,|\, \text{Patientin entwickelt Brustkrebs})=\frac{\#(\text{Brustkrebsdiagnosen bei tatsächlich erkrankten Patientinnen})}{\#(\text{tatsächlich erkrankte Patientinnen})}=\frac{2000}{2700}$$

Den senkrechten Strich in $p(X|Y)$ liest man als "gegeben dass".

**Aufgabe 2** Bestimmen Sie die folgenden (bedingten) Wahrscheinlichkeiten auf dem Wetterausflugs-Datensatz:  
1. $P(Sonne|Ja)=$  
2. $P(Bewölkt|Nein)=$
3. $P(Ja|Regen)=$    

In [8]:
#Lösung 1:
P_gegebendass_Ja = df[df.y=='Ja']
P_gegebendass_Ja

,Aussicht,Temperatur,Feuchtigkeit,Wind,y
2,Bewölkt,Heiss,Hoch,FALSCH,Ja
3,Regen,Mild,Hoch,FALSCH,Ja
4,Bewölkt,Mild,Hoch,FALSCH,Ja
5,Bewölkt,Kühl,Normal,WAHR,Ja
7,Sonne,Mild,Normal,FALSCH,Ja
8,Sonne,Mild,Normal,FALSCH,Ja
9,Regen,Kühl,Normal,WAHR,Ja
10,Regen,Kühl,Normal,WAHR,Ja
12,Bewölkt,Heiss,Normal,FALSCH,Ja


In [9]:
vc = P_gegebendass_Ja['Aussicht'].value_counts()
vc

Bewölkt    4
Regen      3
Sonne      2
Name: Aussicht, dtype: int64

In [10]:
vc.to_dict()['Sonne'],vc.sum(),vc.to_dict()['Sonne']/vc.sum()

(2, 9, 0.222)

also ist P(Sonne|Ja)=22%  


In [11]:
#Lösung 2. P(Bewölkt|y=Nein) = 
P_gegebendass_Nein = df[df.y=='Nein']
P_gegebendass_Nein

,Aussicht,Temperatur,Feuchtigkeit,Wind,y
0,Sonne,Heiss,Hoch,FALSCH,Nein
1,Sonne,Heiss,Hoch,WAHR,Nein
6,Sonne,Mild,Normal,WAHR,Nein
11,Regen,Kühl,Hoch,FALSCH,Nein
13,Regen,Mild,Hoch,WAHR,Nein


P(Bewölkt|y=Nein) = 0/5=0

P(Ja|Aussicht=Regen):

In [12]:
df[df.Aussicht=='Regen']

,Aussicht,Temperatur,Feuchtigkeit,Wind,y
3,Regen,Mild,Hoch,FALSCH,Ja
9,Regen,Kühl,Normal,WAHR,Ja
10,Regen,Kühl,Normal,WAHR,Ja
11,Regen,Kühl,Hoch,FALSCH,Nein
13,Regen,Mild,Hoch,WAHR,Nein


d.h. von den Tagen mit Regen haben drei y=Ja. Daher ist  
$P(Ja|Aussicht=Regen)=3/5=0.6$

## Satz von Bayes

Wir werden gleich den Satz von Bayes verwenden, um eine Vorhersage zu machen. Er lautet allgemein

$$
P(y|X) = \frac{P(X|y) P(y)}{P(X)}
$$

**Aufgabe 3:**
Bestimmen Sie die folgenden Grössen:

a) $P(sonnig)=$  
b) $P(Ja)=$  
c) $P(sonnig|Ja)=$      
d) $P(Ja|sonnig)=$   
e) $P(Nein)=$

In [19]:
#Lösung a:
a = (df['Aussicht']=='Sonne').sum()/df['Aussicht'].size
a

0.357

In [ ]:
(df['Aussicht']).size

In [14]:
#Lösung b:
P_Ja = b = (df.y=='Ja').mean()
b,9/14

(0.643, 0.643)

In [15]:
P_gegeben_Ja = df[(df.y=='Ja')]
P_gegeben_Ja

,Aussicht,Temperatur,Feuchtigkeit,Wind,y
2,Bewölkt,Heiss,Hoch,FALSCH,Ja
3,Regen,Mild,Hoch,FALSCH,Ja
4,Bewölkt,Mild,Hoch,FALSCH,Ja
5,Bewölkt,Kühl,Normal,WAHR,Ja
7,Sonne,Mild,Normal,FALSCH,Ja
8,Sonne,Mild,Normal,FALSCH,Ja
9,Regen,Kühl,Normal,WAHR,Ja
10,Regen,Kühl,Normal,WAHR,Ja
12,Bewölkt,Heiss,Normal,FALSCH,Ja


In [16]:
#Lösung c:
c=(P_gegeben_Ja['Aussicht']=='Sonne').mean()
c,2/9

(0.222, 0.222)

In [17]:
#Lösung d: 
P_gegeben_Sonne = df[df.Aussicht=='Sonne']
P_gegeben_Sonne

,Aussicht,Temperatur,Feuchtigkeit,Wind,y
0,Sonne,Heiss,Hoch,FALSCH,Nein
1,Sonne,Heiss,Hoch,WAHR,Nein
6,Sonne,Mild,Normal,WAHR,Nein
7,Sonne,Mild,Normal,FALSCH,Ja
8,Sonne,Mild,Normal,FALSCH,Ja


In [18]:
d=(P_gegeben_Sonne.y=='Ja').mean()
d,2/5

(0.400, 0.400)

In [19]:
#Lösung e)
P_Nein = (df.y=='Nein').mean()
P_Nein

0.357

In [20]:
P_Ja+P_Nein

1.000

**Aufgabe 4** verifizieren Sie den Satz von Bayes in der Form
$$P(sonnig\,|\,Ja) P(Ja) = P(Ja\,|\,sonnig)P(sonnig),$$ d.h.
$$ c \cdot b = d\cdot a$$
mit a,b,c,d aus Aufgabe 3.

In [21]:
c*b

0.143

In [22]:
d*a

0.143

In [23]:
#c*b==d*a ?? Achtung Rundungsfehler!
c*b - d*a

-0.000

## Vorhersage

Ihre Aufgabe ist es nun, eine Vorhersage zu geben, ob bei den folgenden Bedingungen ein Ausflug wahrscheinlich ist:  
- sonnige Aussichten
- kühle Temperatur
- hohe Feuchtigkeit
- windig

Der Featurevektor $X$ ist also

$$X=(x_1,x_2,x_3,x_4)=(\text{sonnig,kühl,feucht,WAHR}),$$ 
und $$y=\text{Ja}.$$  
Der Satz von Bayes lautet also

$$
P(y \,|\, X) = \frac{P(X \,|\, y) \,P(y)}{P(X)}
$$
Die rechte Seite ist, wie wir gleich sehen werden, einfacher zu berechnen als die linke. Die linke Seite ist aber, was uns interessiert.

Sie müssen nämlich rausfinden, ob es bei gegebenen Wetterbedingungen die Wahrscheinlichkeit, dass der Ausflug statt findet, grösser ist als dass er nicht statt findet: $$P(y=Ja|X) > P(y=Nein|X)$$

Mit dem Satz von Bayes berechnen wir
$$
P(y=Ja|X) = \frac{P(X|y=Ja) P(Ja)}{P(X)}
$$
und 
$$
P(y=Nein|X) = \frac{P(X|y=Nein) P(Nein)}{P(X)}
$$

Die Bedingung $$P(y=Ja|X) > P(y=Nein|X)$$ kann damit umgeschrieben werden zu

$$\frac{P(X|y=Ja) P(Ja)}{P(X)} >\frac{P(X|y=Nein) P(Nein)}{P(X)}$$

Dies bedeutet also $$P(X|y=Ja) P(Ja) >P(X|y=Nein) P(Nein)$$

$P(X)$ hat sich gekürzt- es spielt hier also keine wichtige Rolle.

Es fehlt noch eine vereinfachende Annahme bis wir diese Ungleichung berechnen können.

## Naive Bayes Annahme
Unter "Naive Bayes" versteht man die Annahme, dass die einzelnen Feature voneinander statistisch unabhängig sind. Die Aussage beispielsweise, dass das Feature $x_1$ in einer Klasse $y$ unabhängig ist von dem Feature $x_2$ bedeutet, dass die bedingte Wahrscheinlichkeit $P(x_1|x_2,y)$ gar nicht von $x_2$ abhängt:
$$ P(x_1|x_2,y)=P(x_1|y)$$



Wenn wir also z.B. die Wahrscheinlichkeitsverteilung der Features $x_1,x_2$ innerhalb einer Klasse $y$ berechnen müssen, dann geht das ganz leicht: Diese Grösse $P(x_1,x_2|y)$ kann nämlich umgeformt werden:
$$P(x_1,x_2|y) = P(x_1|x_2,y)P(x_2|y)$$
(Das gilt ganz allgemein, auf Grund der Definition von bedingten Wahrscheinlichkeiten.) Weil nun aber $x_1$ von $x_2$ unabhängig ist, gilt eben $P(x_1|x_2,y)=P(x_1|y)$. Dies setzen wir oben ein:
$$P(x_1,x_2|y) = P(x_1|x_2,y)P(x_2|y)=P(x_1|y)P(x_2|y)$$
Für $N$ Variablen sähe das ganz ähnlich aus:
$$P(x_1,x_2,\ldots,x_N|y)=P(x_1|y)P(x_2|y)\cdots P(x_N|y)$$

Das ist sehr praktisch! Auf der linken Seite der Gleichung können sehr viel mehr unterschiedliche Grössen stehen als rechts. Wir werden viel weniger Trainingsdaten brauchen.
Anstelle der Wahrscheinlichkeitstabelle

| x_1    | x_2  | x_3  | x_4 | y   | $P(\vec{x}|y)$|
|--------|------|------|-----|-----|---------------|
| sonnig | kühl | hoch | WAHR| Ja  |...............|
| sonnig | kühl | tief | WAHR| ?   |...............|
| sonnig | mild | hoch | WAHR| ?   |...............|
| sonnig | mild | tief | WAHR| ?   |...............|
|  ...   | ...  | ...  | ... | ... |...............|

(insgesamt 3\*3\*2\*2\*2-1=71 Zeilen) müssen nun nur noch die folgenden Elemente angegeben werden:

- P(sonnig|Ja), P(bewölkt|Ja)
- P(sonnig|Nein),P(bewölkt|Nein)
- P(kühl|Ja), P(mild|Ja)
- P(kühl|Nein), P(mild|Nein)
- P(hoch|Ja), P(Normal|Ja)
- P(WAHR|Ja), 
- P(WAHR|Nein) 
- P(Ja)

also ((3-1)+(3-1)+(2-1)+(2-1))*2+1=13 Elemente. Beim Elementezählen werden Produkte also zu Summen-- bei vielen Features ist dies ein enormer Effizienzgewinn! Wir brauchen viel weniger Trainingsdaten. Andererseits ist die "naive" Annahme eben auch sehr streng, und passt nicht auf jede Situation. Spielt z.B. die Wortreihenfolge bei der Email-Spamklassifikation wirklich keine Rolle?


**Aufgabe 5:** Berechnen Sie die Grösse $$P(Sonne,kühl,hoch,WAHR|Ja)=P(Sonne|Ja)P(kühl|Ja)P(hoch|Ja)P(WAHR|Ja)$$

a) P(Sonne|Ja)=?  
b) P(kühl|Ja)=?  
c) P(hoch|Ja)=?  
d) P(WAHR|Ja)=?

Schreiben Sie dazu eine Funktion

def PXgivenY(df,Xname,Xval,Yname,Yval):
    ...

welche Ihre bedingten Wahrscheinlichkeiten berechnet.

In [24]:
def PXgivenY(df,Xname,Xval,Yname,Yval):
    """
    Berechnet für die Ereignistabelle df die Grösse P(Xname=Xval|Yname=Yval). 
    Sie wird berechnet als 
    "Anzahl Ereignisse mit Xname=Xval" / "Anzahl Ereignisse mit Yname=Yval".
    """
    m = (df[(df[Xname]==Xval) & (df[Yname]==Yval)]).shape[0]
    n = (df[Yname]==Yval).sum()
    return m/n

PXgivenY(df,'Aussicht','Sonne','y','Ja')

0.222

In [25]:
PXgivenY(df,'Temperatur','Kühl','y','Ja')

0.333

In [26]:
PXgivenY(df,'Feuchtigkeit','Hoch','y','Ja')

0.333

In [27]:
PXgivenY(df,'Wind','WAHR','y','Ja')

0.333

In [28]:
P_x_given_Ja = PXgivenY(df,'Aussicht','Sonne','y','Ja')*PXgivenY(df,'Temperatur','Kühl','y','Ja')\
*PXgivenY(df,'Feuchtigkeit','Hoch','y','Ja')*PXgivenY(df,'Wind','WAHR','y','Ja')
P_x_given_Ja

0.008

In [29]:
P_x_given_Ja- 2/9*3/9*3/9*3/9

0.000

In [30]:
P_x_given_Nein = PXgivenY(df,'Aussicht','Sonne','y','Nein')*PXgivenY(df,'Temperatur','Kühl','y','Nein')\
*PXgivenY(df,'Feuchtigkeit','Hoch','y','Nein')*PXgivenY(df,'Wind','WAHR','y','Nein')
P_x_given_Nein

0.058

In [31]:
P_x_given_Nein - 3/5*1/5*4/5*3/5

-0.000

In [32]:
#Hier unterschlagen wir rechts einen Faktor 1/P(x): 
P_Nein_given_x = P_x_given_Nein*P_Nein
P_Nein_given_x

0.021

In [33]:
#Hier unterschlagen wir rechts ebenfalls einen Faktor 1/P(x): 
P_Ja_given_x = P_x_given_Ja*P_Ja
P_Ja_given_x

0.005

In [34]:
2/9*3/9*3/9*3/9

0.008

In [35]:
P_x_given_Ja

0.008

Damit haben wir unseren Naive-Bayes-Klassifikator:

In [37]:
if P_x_given_Nein > P_x_given_Ja:
    print("Der Ausflug findet hoffentlich nächstes Jahr wieder statt.")
else:
    print("Der Ausflug findet statt!")

Der Ausflug findet hoffentlich nächstes Jahr wieder statt.
